In [1]:
import numpy as np
import mne
import mne_bids
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [4]:
bids_root = '.'
sessions = mne_bids.get_entity_vals(bids_root, 'session', ignore_sessions='on')
bids_paths = mne_bids.find_matching_paths(bids_root, datatypes='eeg', sessions=sessions, extensions='.edf')
bids_paths

TypeError: get_entity_vals() missing 1 required positional argument: 'entity_key'

In [3]:
singleton = bids_paths[0]
raw = mne_bids.read_raw_bids(bids_path=singleton, verbose=True)
raw

IndexError: list index out of range

In [2]:
#read EEG mff data
raw = mne.io.read_raw_egi('sourcedata/s06/eeg/s06/s06_ssvep_20230313_113128.mff/',preload=True)

Reading EGI MFF Header from /home/james/q1k/q1k_eeget_init/sourcedata/s06/eeg/s06/s06_ssvep_20230313_113128.mff...
    Reading events ...
    Assembling measurement info ...
    Synthesizing trigger channel "STI 014" ...
    Excluding events {dstr} ...
Reading 0 ... 445982  =      0.000 ...   445.982 secs...


In [3]:
print(raw)
print(raw.info)

<RawMff | signal1.bin, 145 x 445983 (446.0 s), ~493.6 MB, data loaded>
<Info | 11 non-empty values
 bads: []
 ch_names: E1, E2, E3, E4, E5, E6, E7, E8, E9, E10, E11, E12, E13, E14, ...
 chs: 129 EEG, 16 Stimulus
 custom_ref_applied: True
 device_info: 1 item (dict)
 dig: 132 items (3 Cardinal, 129 EEG)
 highpass: 0.0 Hz
 lowpass: 500.0 Hz
 meas_date: 2023-03-11 11:47:06 UTC
 nchan: 145
 projs: []
 sfreq: 1000.0 Hz
 utc_offset: -0500
>


In [4]:
proc_dat = raw.copy().set_eeg_reference(ref_channels='average')
proc_dat = proc_dat.filter(1., None, fir_design='firwin')
proc_dat = proc_dat.filter(None, 30., fir_design='firwin')

#proc_dat.plot(duration=30, n_channels=64)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 441 samples (0.441 s)



[Parallel(n_jobs=1)]: Done 129 out of 129 | elapsed:    2.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 129 out of 129 | elapsed:    2.8s finished


In [5]:
events = mne.find_events(raw, stim_channel='STI 014')

902 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14]


In [10]:
events[:30]

array([[   28,     0,    10],
       [  533,     0,    14],
       [30339,     0,     2],
       [30372,     0,    11],
       [31752,     0,     1],
       [31872,     0,    12],
       [32005,     0,    11],
       [32173,     0,    12],
       [32338,     0,    11],
       [32506,     0,    12],
       [32671,     0,    11],
       [33256,     0,     2],
       [33288,     0,    11],
       [35068,     0,     3],
       [35188,     0,    12],
       [35238,     0,    11],
       [36573,     0,     4],
       [36604,     0,    11],
       [38087,     0,     3],
       [38205,     0,    12],
       [38254,     0,    11],
       [39589,     0,     5],
       [39621,     0,    11],
       [41302,     0,     1],
       [41422,     0,    12],
       [41554,     0,    11],
       [41723,     0,    12],
       [41888,     0,    11],
       [42056,     0,    12],
       [42221,     0,    11]])

Channels marked as bad:
none


In [74]:
np.unique(events[:,2], axis=0)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [6]:
event_dict = {'visual/steadystate/fq06': 1, 'attention': 2, 'visual/steadystate/fq15': 3,
              'attention': 4, 'attention': 5, 'admin': 6, 'attention': 7, 'break': 8,
              'end': 9, 'photo/bin4': 10, 'photo/bin3': 11, 'photo/bin2': 12, 'photo/bin5': 13,
              'begin': 14}

In [7]:
events_c = events

In [76]:
for i, x in np.ndenumerate(events_c[:,2]):
    if x == 1 or x == 3:
        events_c[i[0]+1, 2] = 15
events_c

array([[    28,      0,     10],
       [   533,      0,     14],
       [ 30339,      0,      2],
       ...,
       [434572,      0,     12],
       [434737,      0,     11],
       [440977,      0,      9]])

In [77]:
#stims=events[np.where(events[:,2] == [2,4])]
#mask = (events_c[:,2] == 15) | (events[:,2] == 4)
mask = (events_c[:,2] == 15)
stims = events_c[mask]
print(len(stims))
stims

120


array([[ 31872,      0,     15],
       [ 35188,      0,     15],
       [ 38205,      0,     15],
       [ 41422,      0,     15],
       [ 44338,      0,     15],
       [ 47555,      0,     15],
       [ 50572,      0,     15],
       [ 53488,      0,     15],
       [ 56804,      0,     15],
       [ 59721,      0,     15],
       [ 62937,      0,     15],
       [ 66153,      0,     15],
       [ 69371,      0,     15],
       [ 72487,      0,     15],
       [ 75704,      0,     15],
       [ 78620,      0,     15],
       [ 81537,      0,     15],
       [ 84654,      0,     15],
       [ 87670,      0,     15],
       [ 90786,      0,     15],
       [ 94102,      0,     15],
       [ 97219,      0,     15],
       [100235,      0,     15],
       [103453,      0,     15],
       [106769,      0,     15],
       [110085,      0,     15],
       [113002,      0,     15],
       [116118,      0,     15],
       [119135,      0,     15],
       [122352,      0,     15],
       [12

In [114]:
iti = np.diff(stims[:,0])
iti

array([ 3316,  3017,  3217,  2916,  3217,  3017,  2916,  3316,  2917,
        3216,  3216,  3218,  3116,  3217,  2916,  2917,  3117,  3016,
        3116,  3316,  3117,  3016,  3218,  3316,  3316,  2917,  3116,
        3017,  3217,  3016,  3016,  3017,  3317,  3116,  2917,  3316,
        3117,  3316,  3117, 22101,  3381,  3016,  3033,  3117,  2900,
        3233,  3200,  3133,  3017,  3316,  3017,  3100,  3232,  3117,
        3217,  3317,  3116,  2917,  3216,  2917,  3016,  2916,  3117,
        3000,  3216,  3033,  3318,  3316,  3017,  2916,  2917,  3216,
        3117,  3316,  3317,  3216,  3316,  2917,  3116, 14950,  2916,
        3117,  3216,  3017,  3116,  3017,  3300,  3216,  3334,  3216,
        2917,  2999,  2934,  3116,  3100,  3334,  3016,  2916,  3217,
        3216,  3216,  3001,  3116,  3133,  3317,  3117,  2899,  3033,
        3117,  3316,  3317,  2917,  3317,  2899,  3333,  3017,  3016,
        2917,  3216])

In [8]:
proc_dat.plot(events=events, duration=30, n_channels=64, color='gray',
         event_color={1: 'r', 2: 'g', 3: 'b', 4: 'm', 5: 'y', 6: 'k', 7: 'k', 8: 'k', 9: 'k', 10: 'k', 11: 'k', 12: 'k', 13: 'k', 14: 'k'})

Using qt as 2D backend.


In [ ]:
reject_criteria = dict(eeg=150e-6)       # 250 µV

epochs = mne.Epochs(raw, events, event_id=event_dict, tmin=-1, tmax=2,
                    reject=reject_criteria, preload=True)

In [80]:
et = mne.io.read_raw_eyelink ('sourcedata/s06/eyetracking/s06ssvep.asc')

Loading sourcedata/s06/eyetracking/s06ssvep.asc
Pixel coordinate data detected.
Pass `scalings=dict(eyegaze=1e3)` when using plot method to make traces more legible.
Pupil-size area reported.
There are 6 recording blocks in this file. Times between blocks will be annotated with bad_rec_gap.


In [82]:
df_et = et.to_data_frame()

In [83]:
df_et

,time,xpos_right,ypos_right,pupil_right,DIN
0,0.000,946.2,558.0,403.0,0.0
1,0.001,946.2,558.5,402.0,0.0
2,0.002,946.4,558.9,402.0,0.0
3,0.003,946.6,558.7,402.0,0.0
4,0.004,946.8,558.9,402.0,0.0
...,...,...,...,...,...
416667,416.667,NaN,NaN,0.0,0.0
416668,416.668,NaN,NaN,0.0,0.0
416669,416.669,NaN,NaN,0.0,0.0
416670,416.670,NaN,NaN,0.0,0.0


In [86]:
df_et['DIN_diff']=df_et['DIN'].diff()
df_et['DIN_diff'].unique()

array([ nan,   0.,   4.,  -4.,   2.,  -2.,  16., -14., -16.,  24., -22.,
       -24.,  20.,  18.])

In [ ]:
.df_et['DIN']unique()

In [88]:
et_stims=df_et.loc[df_et['DIN_diff'].isin([2,4])]
et_stims

,time,xpos_right,ypos_right,pupil_right,DIN,DIN_diff
1023,1.023,940.4,572.6,472.0,4.0,4.0
2523,2.523,956.3,558.9,346.0,2.0,2.0
2656,2.656,954.7,556.5,348.0,4.0,4.0
2824,2.824,947.9,561.7,347.0,2.0,2.0
2989,2.989,948.6,551.7,309.0,4.0,4.0
...,...,...,...,...,...,...
404725,404.725,NaN,NaN,0.0,4.0,4.0
404893,404.893,1664.2,1263.5,54.0,2.0,2.0
405057,405.057,1576.6,922.4,206.0,4.0,4.0
405226,405.226,1331.0,848.6,258.0,2.0,2.0


In [98]:
et_stims_ri = et_stims.reset_index()
et_stims_ri

,index,time,xpos_right,ypos_right,pupil_right,DIN,DIN_diff
0,1023,1.023,940.4,572.6,472.0,4.0,4.0
1,2523,2.523,956.3,558.9,346.0,2.0,2.0
2,2656,2.656,954.7,556.5,348.0,4.0,4.0
3,2824,2.824,947.9,561.7,347.0,2.0,2.0
4,2989,2.989,948.6,551.7,309.0,4.0,4.0
...,...,...,...,...,...,...,...
1296,404725,404.725,NaN,NaN,0.0,4.0,4.0
1297,404893,404.893,1664.2,1263.5,54.0,2.0,2.0
1298,405057,405.057,1576.6,922.4,206.0,4.0,4.0
1299,405226,405.226,1331.0,848.6,258.0,2.0,2.0


In [109]:
len(et_stims_ri)

1301

In [111]:
for ind, row in et_stims_ri.iterrows():
    if et_stims_ri['DIN_diff'][ind] == 4:
        if ind < len(et_stims_ri)-1:
            if et_stims_ri['DIN_diff'][ind+1] == 4:
                et_stims_ri['DIN_diff'][ind+1] = 5



/tmp/ipykernel_57620/615722596.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [112]:
et_stims_rit=et_stims_ri.loc[et_stims_ri['DIN_diff'].isin([5])]
et_stims_rit

,index,time,xpos_right,ypos_right,pupil_right,DIN,DIN_diff
7,3939,3.939,941.5,531.1,207.0,4.0,5.0
22,7256,7.256,938.9,531.3,196.0,4.0,5.0
37,10272,10.272,947.0,529.0,221.0,4.0,5.0
44,13489,13.489,932.2,530.2,277.0,4.0,5.0
59,16405,16.405,939.3,544.8,275.0,4.0,5.0
...,...,...,...,...,...,...,...
1252,392991,392.991,936.4,552.1,220.0,4.0,5.0
1257,393841,393.841,931.2,547.5,200.0,4.0,5.0
1272,396858,396.858,958.7,467.6,170.0,4.0,5.0
1287,399874,399.874,929.3,538.6,177.0,4.0,5.0


In [113]:
d=et_stims_rit['index'].diff()
d

7          NaN
22      3317.0
37      3016.0
44      3217.0
59      2916.0
         ...  
1252    2466.0
1257     850.0
1272    3017.0
1287    3016.0
1294    2917.0
Name: index, Length: 129, dtype: float64

In [117]:
fig = px.line(
                y=d,
                )
fig.show()

In [118]:
fig = px.line(
                y=iti,
                )
fig.show()

In [115]:
iti

array([ 3316,  3017,  3217,  2916,  3217,  3017,  2916,  3316,  2917,
        3216,  3216,  3218,  3116,  3217,  2916,  2917,  3117,  3016,
        3116,  3316,  3117,  3016,  3218,  3316,  3316,  2917,  3116,
        3017,  3217,  3016,  3016,  3017,  3317,  3116,  2917,  3316,
        3117,  3316,  3117, 22101,  3381,  3016,  3033,  3117,  2900,
        3233,  3200,  3133,  3017,  3316,  3017,  3100,  3232,  3117,
        3217,  3317,  3116,  2917,  3216,  2917,  3016,  2916,  3117,
        3000,  3216,  3033,  3318,  3316,  3017,  2916,  2917,  3216,
        3117,  3316,  3317,  3216,  3316,  2917,  3116, 14950,  2916,
        3117,  3216,  3017,  3116,  3017,  3300,  3216,  3334,  3216,
        2917,  2999,  2934,  3116,  3100,  3334,  3016,  2916,  3217,
        3216,  3216,  3001,  3116,  3133,  3317,  3117,  2899,  3033,
        3117,  3316,  3317,  2917,  3317,  2899,  3333,  3017,  3016,
        2917,  3216])

In [95]:
fig = px.line(
                y=et_stims['DIN_diff'],
                #y='DIN',
                #opacity=.5,
                #marginal_y="histogram",
                #marginal_x="histogram"
                )
fig.show()
